# Data Cleaning & Preparation

> Load and Profile Data

In [10]:
import pandas as pd

# Load data
file_path = 'Senior Data Analyst Task Transactions.csv'
df = pd.read_csv(file_path)

In [11]:
df.shape

(10000, 12)

In [12]:
df.columns

Index(['Transaction_ID', 'User_ID', 'Crypto', 'Transaction_Type', 'Amount',
       'Price_Per_Unit', 'Total_Value', 'Transaction_Fee', 'Platform',
       'Status', 'Wallet_Type', 'Transaction_Date'],
      dtype='object')

In [13]:
df.head()

,Transaction_ID,User_ID,Crypto,Transaction_Type,Amount,Price_Per_Unit,Total_Value,Transaction_Fee,Platform,Status,Wallet_Type,Transaction_Date
0,434465,31394,Bitcoin,Withdraw,57.695145,45248.78,2610634.92,114990.70,KuCoin,Completed,Trezor,2024-04-10 09:51:58.222424
1,905397,39627,Solana,Stake,27.708982,48307.43,1338549.71,34560.11,Binance,Failed,Cold Wallet,2023-10-04 09:51:58.222424
2,451293,11506,Cardano,Buy,80.368977,44428.82,3570698.81,38312.76,OKX,Pending,Hot Wallet,2024-11-15 09:51:58.222424
3,841249,84005,XRP,Stake,83.900483,8755.31,734574.74,1396.89,FTX,Completed,Exchange Wallet,2024-02-11 09:51:58.222424
4,762686,66706,Dogecoin,Stake,52.528368,58667.26,3081695.42,19087.58,Kraken,Pending,Exchange Wallet,2023-09-15 09:51:58.222424


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction_ID    10000 non-null  int64  
 1   User_ID           10000 non-null  int64  
 2   Crypto            10000 non-null  object 
 3   Transaction_Type  10000 non-null  object 
 4   Amount            10000 non-null  float64
 5   Price_Per_Unit    10000 non-null  float64
 6   Total_Value       10000 non-null  float64
 7   Transaction_Fee   10000 non-null  float64
 8   Platform          10000 non-null  object 
 9   Status            10000 non-null  object 
 10  Wallet_Type       10000 non-null  object 
 11  Transaction_Date  10000 non-null  object 
dtypes: float64(4), int64(2), object(6)
memory usage: 937.6+ KB


In [15]:
df.describe(include='all')

,Transaction_ID,User_ID,Crypto,Transaction_Type,Amount,Price_Per_Unit,Total_Value,Transaction_Fee,Platform,Status,Wallet_Type,Transaction_Date
count,10000.000000,10000.00000,10000,10000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000,10000,10000,10000
unique,NaN,NaN,10,5,NaN,NaN,NaN,NaN,7,3,5,731
top,NaN,NaN,Polkadot,Stake,NaN,NaN,NaN,NaN,Coinbase,Failed,Hot Wallet,2024-10-29 09:51:58.222424
freq,NaN,NaN,1077,2052,NaN,NaN,NaN,NaN,1515,3353,2064,27
mean,547132.392100,55176.38050,NaN,NaN,50.120755,29888.220522,1.506658e+06,37555.080850,NaN,NaN,NaN,NaN
std,262163.276431,25823.90504,NaN,NaN,28.902577,17215.202926,1.327017e+06,43871.976036,NaN,NaN,NaN,NaN
min,100027.000000,10016.00000,NaN,NaN,0.024363,10.270000,3.665000e+01,0.040000,NaN,NaN,NaN,NaN
25%,318120.750000,32775.00000,NaN,NaN,24.889845,15240.167500,4.124107e+05,6007.977500,NaN,NaN,NaN,NaN
50%,547635.000000,55643.00000,NaN,NaN,49.887958,29823.745000,1.124990e+06,21182.080000,NaN,NaN,NaN,NaN
75%,775336.000000,77450.00000,NaN,NaN,75.319944,44728.790000,2.310646e+06,53563.732500,NaN,NaN,NaN,NaN


> Data Profiling

In [16]:
!pip install ydata_profiling

In [9]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Crypto Transactions Profiling Report", explorative=True)
profile.to_file("crypto_transactions_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 12/12 [00:00<00:00, 16.37it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Data Cleaning

In [17]:
# Handle missing values
df_clean = df.copy()
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        df_clean[col].fillna('Unknown', inplace=True)
    else:
        df_clean[col].fillna(df_clean[col].median(), inplace=True)

# Drop duplicates
df_clean.drop_duplicates(inplace=True)

# Convert date columns (assume column is named 'date')
if 'date' in df_clean.columns:
    df_clean['date'] = pd.to_datetime(df_clean['date'], errors='coerce')

# Save cleaned CSV for dashboards
df_clean.to_csv('cleaned_crypto_transactions.csv', index=False)
df_clean

<ipython-input-17-8dd07966d148>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(df_clean[col].median(), inplace=True)
<ipython-input-17-8dd07966d148>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

,Transaction_ID,User_ID,Crypto,Transaction_Type,Amount,Price_Per_Unit,Total_Value,Transaction_Fee,Platform,Status,Wallet_Type,Transaction_Date
0,434465,31394,Bitcoin,Withdraw,57.695145,45248.78,2610634.92,114990.70,KuCoin,Completed,Trezor,2024-04-10 09:51:58.222424
1,905397,39627,Solana,Stake,27.708982,48307.43,1338549.71,34560.11,Binance,Failed,Cold Wallet,2023-10-04 09:51:58.222424
2,451293,11506,Cardano,Buy,80.368977,44428.82,3570698.81,38312.76,OKX,Pending,Hot Wallet,2024-11-15 09:51:58.222424
3,841249,84005,XRP,Stake,83.900483,8755.31,734574.74,1396.89,FTX,Completed,Exchange Wallet,2024-02-11 09:51:58.222424
4,762686,66706,Dogecoin,Stake,52.528368,58667.26,3081695.42,19087.58,Kraken,Pending,Exchange Wallet,2023-09-15 09:51:58.222424
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,186998,93233,Bitcoin,Stake,30.918047,20067.21,620438.94,5603.47,FTX,Pending,Trezor,2023-02-28 09:51:58.222424
9996,337144,44181,Cardano,Buy,80.090849,42488.69,3402955.25,71081.12,Kraken,Failed,Ledger,2024-08-28 09:51:58.222424
9997,946655,22496,BNB,Withdraw,19.005921,36513.00,693963.19,29738.11,Coinbase,Completed,Ledger,2023-04-06 09:51:58.222424
9998,153921,72835,Litecoin,Buy,11.872668,19748.07,234462.28,10107.57,OKX,Failed,Trezor,2023-11-01 09:51:58.222424


# Summary Stats & Trends

In [18]:
# Summary statistics
summary = {
    'total_transactions': len(df_clean),
    'unique_assets': df_clean['asset'].nunique() if 'asset' in df_clean.columns else None,
    'unique_wallets': df_clean['wallet'].nunique() if 'wallet' in df_clean.columns else None,
    'total_volume': df_clean['amount'].sum() if 'amount' in df_clean.columns else None,
    'median_txn_size': df_clean['amount'].median() if 'amount' in df_clean.columns else None,
}

print(summary)

{'total_transactions': 10000, 'unique_assets': None, 'unique_wallets': None, 'total_volume': None, 'median_txn_size': None}


# Transaction Volume Over Time (Line Chart)

In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load data
df = pd.read_csv('cleaned_crypto_transactions.csv')

# Convert date column
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])

# --- PLOT 1: Transaction Volume Over Time (USD) ---
daily = df.groupby(df['Transaction_Date'].dt.date)['Total_Value'].sum().reset_index()
fig_line = px.line(
    daily, x='Transaction_Date', y='Total_Value',
    title='Transaction Volume (USD) Over Time',
    labels={'Total_Value': 'Total Volume (USD)', 'Transaction_Date': 'Date'}
)
fig_line.show()

# PLOT 2: Top Assets by Transaction Value

In [20]:

asset_totals = df.groupby('Crypto')['Total_Value'].sum().sort_values(ascending=False).head(10)
fig_bar = px.bar(
    x=asset_totals.index, y=asset_totals.values,
    title='Top 10 Crypto Assets by Transaction Value (USD)',
    labels={'x': 'Crypto Asset', 'y': 'Total Transaction Value (USD)'}
)
fig_bar.show()

# PLOT 3: Distribution of Transaction Amounts

In [21]:
fig_hist = px.histogram(
    df, x='Total_Value', nbins=50,
    title='Distribution of Transaction Values (USD)',
    labels={'Total_Value': 'Transaction Value (USD)'}
)
fig_hist.show()

# KPI SUMMARY TABLE

In [22]:
summary = {
    'Total Transactions': [len(df)],
    'Unique Crypto Assets': [df['Crypto'].nunique()],
    'Unique Users': [df['User_ID'].nunique()],
    'Total Volume (USD)': [df['Total_Value'].sum()],
    'Median Transaction Value (USD)': [df['Total_Value'].median()]
}

fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(summary.keys())),
    cells=dict(values=[v for v in summary.values()])
)])
fig_table.show()

# Outliers

In [23]:
# Define outlier threshold using IQR
Q1 = df['Total_Value'].quantile(0.25)
Q3 = df['Total_Value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Total_Value'] < lower_bound) | (df['Total_Value'] > upper_bound)]
print(f"Outlier transactions based on Total_Value ({len(outliers)} found):")
display(outliers[['Transaction_ID', 'Total_Value', 'Crypto', 'Transaction_Date']].head())

Outlier transactions based on Total_Value (91 found):


,Transaction_ID,Total_Value,Crypto,Transaction_Date
17,206002,5280448.80,BNB,2023-12-14 09:51:58.222424
31,183990,5425132.22,XRP,2025-02-24 09:51:58.222424
101,502571,5160767.47,BNB,2024-03-01 09:51:58.222424
115,720443,5302247.61,Polkadot,2023-06-12 09:51:58.222424
145,852333,5483533.32,XRP,2024-11-14 09:51:58.222424


In [24]:
# Outlier detection using IQR for Total_Value
Q1 = df['Total_Value'].quantile(0.25)
Q3 = df['Total_Value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Total_Value'] < lower_bound) | (df['Total_Value'] > upper_bound)]
normal = df[(df['Total_Value'] >= lower_bound) & (df['Total_Value'] <= upper_bound)]
hover_cols = ['Transaction_ID', 'Crypto', 'Platform', 'Transaction_Type', 'Transaction_Date']

normal_customdata = normal[hover_cols].values
outliers_customdata = outliers[hover_cols].values

fig_scatter = go.Figure()
fig_scatter.add_trace(go.Scatter(
    x=normal['Total_Value'],
    y=normal['Transaction_Fee'],
    mode='markers',
    name='Normal',
    marker=dict(color='blue', opacity=0.5),
    customdata=normal_customdata,
    hovertemplate=
        "<b>Total Value</b>: %{x}<br>"+
        "<b>Transaction Fee</b>: %{y}<br>"+
        "<b>Transaction ID</b>: %{customdata[0]}<br>"+
        "<b>Crypto</b>: %{customdata[1]}<br>"+
        "<b>Platform</b>: %{customdata[2]}<br>"+
        "<b>Type</b>: %{customdata[3]}<br>"+
        "<b>Date</b>: %{customdata[4]}<br>"+
        "<extra></extra>"
))
fig_scatter.add_trace(go.Scatter(
    x=outliers['Total_Value'],
    y=outliers['Transaction_Fee'],
    mode='markers',
    name='Outliers',
    marker=dict(color='red', size=10, symbol='x'),
    customdata=outliers_customdata,
    hovertemplate=
        "<b>Total Value</b>: %{x}<br>"+
        "<b>Transaction Fee</b>: %{y}<br>"+
        "<b>Transaction ID</b>: %{customdata[0]}<br>"+
        "<b>Crypto</b>: %{customdata[1]}<br>"+
        "<b>Platform</b>: %{customdata[2]}<br>"+
        "<b>Type</b>: %{customdata[3]}<br>"+
        "<b>Date</b>: %{customdata[4]}<br>"+
        "<extra></extra>"
))
fig_scatter.update_layout(
    title='Transaction Value vs Transaction Fee (Outliers Highlighted)',
    xaxis_title='Total Value (USD)',
    yaxis_title='Transaction Fee (USD)',
    legend_title='Transaction Type'
)
fig_scatter.show()